In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from operator import itemgetter

In [2]:
# Ваша задача – найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". 
# Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.
# В качестве ответа выберите id объектов (3), которые присутствуют в IDs_for_answer.

In [202]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/search?q='
IDs = 'sunflowers'

In [203]:
r = requests.get(url+IDs) # создаем запрос из url поиска и подсолнухов
res = r.json()

In [204]:
sunflower_ids = res['objectIDs']

In [6]:
IDs_for_answer = [20149, 437329, 2032, 16822570, 436524]

In [7]:
for i in sunflower_ids:
    if i in IDs_for_answer:
        print(i)

436524
2032
437329


In [8]:
# Следущий этап – собрать информацию об этих объектах. 
# Выполните запрос для объекта 437980. Для проверки соотнесите полученные параметры с ключами:
# title', 'department', 'GalleryNumber', 'culture', 'objectID', 'artistDisplayName', 'objectName'

In [9]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_437980 = r_test.json()

In [10]:
obj_437980_params = ['title', 'department', 'GalleryNumber', 'culture', 'objectID', 'artistDisplayName', 'objectName']

In [11]:
itemgetter('title', 'department', 'GalleryNumber', 'culture', 'objectID', 'artistDisplayName', 'objectName')(obj_437980)
# itemgetter из operator позволяет обратиться к словарю по нескольким ключам

('Cypresses',
 'European Paintings',
 '825',
 '',
 437980,
 'Vincent van Gogh',
 'Painting')

In [53]:
# Теперь напишите цикл, с помощью которого будет собрана информация об объектах sunflower_ids 
# Результаты запишите в датафрейм

In [164]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

In [178]:
df = pd.DataFrame()

In [179]:
for i in sunflower_ids:
    req = requests.get(f'{url}{i}')
    res = req.json()
    try:
        tmp_df = pd.DataFrame({'object_id': [res['objectID']], 'culture': [res['culture']]})
        df = pd.concat([df, tmp_df])
    except:
        continue

In [187]:
df = df.reset_index(drop=True)

In [189]:
df.culture.unique()

array(['', 'American', 'Japan', 'British', 'Ancestral Pueblo'],
      dtype=object)

In [194]:
df.query('culture == "Japan"')

,object_id,culture
29,57922,Japan


In [267]:
# Теперь попробуем построить более сложный запрос с помощью секции "Search"! 
# На этот раз параметры необходимо передать через "&", например, departmentId (id департамента – 1) и q (ключевое слово – cat)
# Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. 
# Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

# В качестве ответа укажите, в каком периоде (period) было выполнено наибольшее количество работ, 
# представленных в музее?

In [268]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat'

In [269]:
r = requests.get(f'{url}')
req = r.json()

In [270]:
cat_ids = req['objectIDs']

In [271]:
df_cats = pd.DataFrame()

In [272]:
for i in cat_ids[0:1000]:
    req = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
    res = req.json()
    tmp_df = pd.DataFrame({'object_id': [res['objectID']], 'period': [res['period']]})
    df_cats = pd.concat([df_cats, tmp_df])

In [273]:
df_cats = df_cats.reset_index().drop('index', 1)

In [279]:
df_cats.period.value_counts().idxmax()

'Edo period (1615–1868)'